In [2]:
import boto3
import json
from jinja2 import Environment, FileSystemLoader

In [3]:
bedrock_runtime= boto3.client('bedrock-runtime',region_name='us-east-1')

In [4]:
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("deep_insight_prompt.txt")

In [5]:
passage='The quick brown fox jumps over the lazy dog.'

In [6]:
prompt = template.render(
    passage=passage
)

In [7]:
prompt

'You are a faithful and trusted network graph maker tasked with creating a comprehensive and interconnected representation of a given context (input text), ensuring all nodes are part of a single cluster under a parent node called \'Knowledge\'. Your job involves extracting key concepts (nodes) and their relationships (links) from the provided text.\n\nWhile doing so, follow these nine guidelines below faithfully:\n\n1. Nodes Identification: Identify key nodes mentioned in the text, including objects, entitites, locations, organizations, persons, conditions, acronyms, documents, services, concepts, etc. Ensure nodes are atomic and clearly represent distinct aspects of the context.\n\n2. Relationship Mapping: Establish clear, one-to-one relationships between all nodes. Nodes in proximity (same sentance or paragraph) likely share a relationship. Ensure every node is connected to at least one other node, preventing isolated clusters.\n\n3. Relationship validation: Confirm each relationshi

In [11]:
body=json.dumps(
{
    'prompt':prompt,
    'temperature':0.5,
    'top_p':0.5,
    'max_gen_len':1000
})

In [12]:
body

'{"prompt": "You are a faithful and trusted network graph maker tasked with creating a comprehensive and interconnected representation of a given context (input text), ensuring all nodes are part of a single cluster under a parent node called \'Knowledge\'. Your job involves extracting key concepts (nodes) and their relationships (links) from the provided text.\\n\\nWhile doing so, follow these nine guidelines below faithfully:\\n\\n1. Nodes Identification: Identify key nodes mentioned in the text, including objects, entitites, locations, organizations, persons, conditions, acronyms, documents, services, concepts, etc. Ensure nodes are atomic and clearly represent distinct aspects of the context.\\n\\n2. Relationship Mapping: Establish clear, one-to-one relationships between all nodes. Nodes in proximity (same sentance or paragraph) likely share a relationship. Ensure every node is connected to at least one other node, preventing isolated clusters.\\n\\n3. Relationship validation: Conf

In [17]:
response = bedrock_runtime.invoke_model(
    modelId='meta.llama2-70b-chat-v1',
    contentType='application/json',
    accept='application/json',
    body=body
)

In [18]:
response_body = json.loads(response['body'].read())

In [19]:
response_body

{'generation': '\n\nOutput:\n\n{\n"nodes": [\n{\n"id": 0,\n"type": "concept",\n"group": 1\n},\n{\n"id": 1,\n"type": "animal",\n"group": 2\n},\n{\n"id": 2,\n"type": "animal",\n"group": 2\n},\n{\n"id": 3,\n"type": "action",\n"group": 3\n},\n{\n"id": 4,\n"type": "object",\n"group": 4\n}\n],\n"links": [\n{\n"source": 0,\n"target": 1,\n"label": "is-a",\n"strength": 0.8,\n"rationale": "The quick brown fox is an animal."\n},\n{\n"source": 0,\n"target": 2,\n"label": "is-a",\n"strength": 0.8,\n"rationale": "The lazy dog is an animal."\n},\n{\n"source": 1,\n"target": 3,\n"label": "performs",\n"strength": 0.7,\n"rationale": "The quick brown fox jumps over the lazy dog."\n},\n{\n"source": 2,\n"target": 3,\n"label": "is-a-target-of",\n"strength": 0.6,\n"rationale": "The lazy dog is the target of the jump."\n},\n{\n"source": 3,\n"target": 4,\n"label": "involves",\n"strength": 0.5,\n"rationale": "The jump involves the fox and the dog."\n}\n],\n"groups": [\n{\n"group_id": 1,\n"rationale": "All concept

In [20]:
with open("original_prompt_output/llama2-70b-quick-fox-1k-tokens.json","w+") as f:
    json.dump(response_body,f)